# Titanic- Who will survive?
***

1. Explore the features 
2. Apply feature engineering
3. Build models using code in src/
4. Evaluate the models
5. Interpret the results
6. Answer some questions

In [ ]:
import pandas as pd # for loading and performing analysis on the dataset
